In [2]:
import requests
import re
import pandas as pd
from string import digits
from bs4 import BeautifulSoup

In [3]:
url = "http://www.banki.ru/services/responses/list/"
response = requests.get(url)
i = 1
soup = BeautifulSoup(response.text, "html5lib")

In [4]:
reviews = soup.find_all("article", attrs={"class": "responses__item"})
df = pd.DataFrame(columns = ('date', 'bank', 'title', 'rating', 'text'), index = [0])

в цикле переходим между страницами, на каждой странице проверяем каждый отзыв, при возможности считываем полный отзыв по ссылке. оцищаем от пунктуации и лишних пробелов. пишем все в DataFrame

In [5]:
while (response and i < 400) :
    for review in reviews:
    # дата
        date = review.find("time", "display-inline-block").text

    # название банка
        bank_name = review.find_all("a", attrs={"class": "responses__item__logo lazy-load"})[0].get("title")
        bank_name = re.sub(' +',' ', bank_name)
        bank_name = re.sub('[A-Za-z0-9]+\s+', '', bank_name)

    # заголовок
        title = review.find_all("a", attrs={"class": "font-size-large font-bold"})[0].text
        title = re.sub(' +',' ', title)
        title = re.sub('[A-Za-z0-9]+\s+', '', title)    

    # рейтинг
        rating = review.find("div", attrs={"class": "responses__item__rating"}).find("meta")
        if rating: rating = rating.get("content")
        else: rating = None

    # полный текст отзыва
        link = review.find("div", attrs={"responses__item__message"})
        if link is not None:
            if (link.find("a", href=True) is not None):
                url1 = "http://www.banki.ru" + link.find("a", href=True)['href']
                text_page = requests.get(url1)
                soup1 = BeautifulSoup(text_page.text, "html5lib")
                text = soup1.find("div", "article-text response-page__text markup-inside-small markup-inside-small--bullet")
                if (text is not None): text = text.text 
                else: text = review.find("div", "responses__item__message").text
            else: text = review.find("div", "responses__item__message").text 
        else: text = review.find("div", "responses__item__message").text
        text = re.sub(' +',' ', text)
        text = re.sub('[A-Za-z0-9]+\s+', '', text)
        remove_digits = str.maketrans('', '', digits)
        text = text.translate(remove_digits).lstrip().rstrip()

    # запись в DataFrame
        df = df.append({
            'date': date,
            'bank': bank_name,
            'title': title,
            'rating': rating,
            'text': text[:150]
        }, ignore_index=True)
    
    # листаем по страницам
    i+=1
    url2 = url+"?page="+str(i)
    response = requests.get(url2)
    soup = BeautifulSoup(response.text, "html5lib")
    reviews = soup.find_all("article", attrs={"class": "responses__item"})
    print (url2)

http://www.banki.ru/services/responses/list/?page=2
http://www.banki.ru/services/responses/list/?page=3
http://www.banki.ru/services/responses/list/?page=4
http://www.banki.ru/services/responses/list/?page=5
http://www.banki.ru/services/responses/list/?page=6
http://www.banki.ru/services/responses/list/?page=7
http://www.banki.ru/services/responses/list/?page=8
http://www.banki.ru/services/responses/list/?page=9
http://www.banki.ru/services/responses/list/?page=10
http://www.banki.ru/services/responses/list/?page=11
http://www.banki.ru/services/responses/list/?page=12
http://www.banki.ru/services/responses/list/?page=13
http://www.banki.ru/services/responses/list/?page=14
http://www.banki.ru/services/responses/list/?page=15
http://www.banki.ru/services/responses/list/?page=16
http://www.banki.ru/services/responses/list/?page=17
http://www.banki.ru/services/responses/list/?page=18
http://www.banki.ru/services/responses/list/?page=19
http://www.banki.ru/services/responses/list/?page=20
h

http://www.banki.ru/services/responses/list/?page=156
http://www.banki.ru/services/responses/list/?page=157
http://www.banki.ru/services/responses/list/?page=158
http://www.banki.ru/services/responses/list/?page=159
http://www.banki.ru/services/responses/list/?page=160
http://www.banki.ru/services/responses/list/?page=161
http://www.banki.ru/services/responses/list/?page=162
http://www.banki.ru/services/responses/list/?page=163
http://www.banki.ru/services/responses/list/?page=164
http://www.banki.ru/services/responses/list/?page=165
http://www.banki.ru/services/responses/list/?page=166
http://www.banki.ru/services/responses/list/?page=167
http://www.banki.ru/services/responses/list/?page=168
http://www.banki.ru/services/responses/list/?page=169
http://www.banki.ru/services/responses/list/?page=170
http://www.banki.ru/services/responses/list/?page=171
http://www.banki.ru/services/responses/list/?page=172
http://www.banki.ru/services/responses/list/?page=173
http://www.banki.ru/services

ConnectionError: ('Connection aborted.', TimeoutError(10060, 'A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond', None, 10060, None))

In [ ]:
df[1:].to_csv('data.csv')

In [ ]:
df.iloc[0:0]